In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from script.gpt import GPT
from script.gpt_utils import beautiful_print

In [2]:
with open('data/truyen_kieu_clean.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

print("Vocab size:", vocab_size)
print("Number of characters:", len(text))

Vocab size: 129
Number of characters: 104804


In [3]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encoder = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decoder = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

test_text = """
Trăm năm trong cõi người ta,
Chữ tài chữ mệnh khéo là ghét nhau.
Trải qua một cuộc bể dâu,
Những điều trông thấy mà đau đớn lòng.
"""
print(encode(test_text))
print(decode(encode(test_text)))

NameError: name 'encode' is not defined

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_train_batch(train_data, batch_size:int = 64, block_size: int = 32):
    idx = torch.randint(len(train_data) - block_size, (batch_size, ))
    x = torch.stack([train_data[i:i+block_size] for i in idx])
    y = torch.stack([train_data[i+1:i+block_size+1] for i in idx])
    return x, y

In [ ]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.enable_flash_sdp(True)

print(torch.backends.cuda.matmul.allow_tf32)  # Should print True
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.__version__)  # Check your PyTorch version
print(torch.version.cuda)  # Check the CUDA version PyTorch was built with
print(torch.backends.cudnn.version())  # Check the cuDNN version